In [ ]:
import cv2
import numpy as np


def cv_imread(file_path: str):
    """
    兼容中文路径的 imread
    """
    data = np.fromfile(file_path, dtype=np.uint8)
    img = cv2.imdecode(data, cv2.IMREAD_COLOR)
    return img


def compute_focus_map(
    img_bgr: np.ndarray,
    blur_sigma: float = 2.0,
    percentile: float = 0.85,
) -> np.ndarray:
    """
    使用 Laplacian 计算清晰度热力图（0~1），并做平滑 + 高亮增强。

    返回:
        focus_norm: float32, [H, W]，0~1，值越大越清晰
    """
    # 转灰度并归一化
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray_f = gray.astype(np.float32) / 255.0

    # Laplacian 边缘 / 清晰度响应
    lap = cv2.Laplacian(gray_f, cv2.CV_32F, ksize=3)
    focus = np.abs(lap)

    # 高斯平滑，去噪 & 保证热力图平滑
    focus = cv2.GaussianBlur(focus, (0, 0), sigmaX=blur_sigma, sigmaY=blur_sigma)

    # 归一化到 0~1
    focus_norm = cv2.normalize(focus, None, 0.0, 1.0, cv2.NORM_MINMAX)

    # 只增强上百分之 (1 - percentile) 最清晰区域
    thr = np.quantile(focus_norm, percentile)
    focus_norm = np.clip((focus_norm - thr) / (1.0 - thr + 1e-6), 0.0, 1.0)

    return focus_norm.astype(np.float32)


def overlay_focus_heatmap(
    img_bgr: np.ndarray,
    focus_map: np.ndarray,
    max_alpha: float = 0.8,
) -> np.ndarray:
    """
    将焦点热力图以红色高亮形式叠加到原图上（BGR）。

    参数:
        img_bgr: 原图 (H,W,3) uint8
        focus_map: 0~1 清晰度图 (H,W) float32
        max_alpha: 最强区域的最大叠加透明度（0~1）

    返回:
        overlay: 叠加后的 BGR 图像，uint8
    """
    base = img_bgr.astype(np.float32)
    h, w = base.shape[:2]
    focus = cv2.resize(focus_map, (w, h), interpolation=cv2.INTER_LINEAR)

    # 构造红色热力图（BGR，R 通道随清晰度变化）
    heat = np.zeros_like(base, dtype=np.float32)
    # BGR 中 R 通道是 index=2
    heat[..., 2] = focus * 255.0

    # 透明度随清晰度变化，最高 max_alpha
    alpha = (focus * max_alpha).astype(np.float32)
    alpha_3 = alpha[..., None]

    overlay = base * (1.0 - alpha_3) + heat * alpha_3
    overlay = np.clip(overlay, 0, 255).astype(np.uint8)
    return overlay


def focus_peaking_thumbnail(
    img_path: str,
    max_width: int = 640,
    blur_sigma: float = 2.0,
    percentile: float = 0.85,
    max_alpha: float = 0.8,
):
    """
    读取原图 -> 生成缩略图 -> 计算焦点热力图 -> 叠加可视化

    返回:
        thumb: 缩略图 BGR uint8
        overlay: 叠加焦点热力图后的缩略图 BGR uint8
        focus_map: 缩略图上的 0~1 清晰度图 float32
    """
    img = cv_imread(img_path)
    if img is None:
        raise ValueError(f"Failed to read image: {img_path}")

    h, w = img.shape[:2]
    if w > max_width:
        scale = max_width / w
        thumb = cv2.resize(
            img,
            (int(w * scale), int(h * scale)),
            interpolation=cv2.INTER_AREA,
        )
    else:
        thumb = img.copy()

    focus_map = compute_focus_map(thumb, blur_sigma=blur_sigma, percentile=percentile)
    overlay = overlay_focus_heatmap(thumb, focus_map, max_alpha=max_alpha)

    return thumb, overlay, focus_map


if __name__ == "__main__":
    # === 在 640.jpg 上测试 ===
    thumb, overlay, focus_map = focus_peaking_thumbnail("640.jpg")

    # 保存到本地查看效果
    cv2.imwrite("640_thumb.jpg", thumb)
    cv2.imwrite("640_focus_overlay.jpg", overlay)
    # 若需要单独看热力图，可以把其拉成 0~255 灰度图保存
    focus_vis = (focus_map * 255).astype(np.uint8)
    cv2.imwrite("640_focus_map.jpg", focus_vis)


: 